In [1]:
#importing the required packages
import requests
import time
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import datetime

In [2]:
def retrieving_information_for_the_day(year,month,day):
  # iterate the dates to get news from these dates
  web_url= "https://www.thehindu.com/archive/web/year/month/day/"
  #getting html source code from the url
  raw_data=requests.get(web_url,'lxml')
  #We create a Beautiful soup object and use it to find the required information from the url
  soup=bs(raw_data.content,'html5lib')
  information=soup.findAll('ul',class_='archive-list')
  return information

In [3]:
#the headlines from the day
def headlines(information):
  headlines=[]
  for i in range(len(information)):
      for j in range(len(information[i].findAll('a'))):
          headlines.append(information[i].findAll('a')[j])
  return headlines

In [4]:
#this is a function to scrap the title of the news and the link to the main content from the information
def links(headlines):
    links=[]
    for i in range(len(headlines)):
        links.append(headlines[i]['href'])
    return links

In [5]:
#this function scraps the authors of the articles and  identifies the articles with no authors.
#These articles are then removed.
def authors(links):
    authors=[]
    noauthors=[]
    c=0
    for i in range(len(links)):
        content_data=requests.get(links[i],'lxml')
        soup1=bs(content_data.content,'html5lib')
        author=soup1.find('a',{"class":["auth-nm lnk","auth-nm no-lnk"]})
        if author!=None:
            authors.append(author.get_text().strip('\n'))
        if author==None:
            noauthors.append(i)
            authors.append(0)
    return authors,noauthors

In [ ]:
def processing(authors,noauthors,links,headlines):
    c=0
    for i in noauthors:
        authors.remove(authors[i-c])
        links.remove(links[i-c])
        headlines.remove(headlines[i-c])
        c=c+1
    return authors,links,headlines

In [ ]:
#this is a function to scrap the title of the news and the link to the main content from the information
def news_titles(headlines):
    news_titles=[]
    for i in range(len(headlines)):
        news_titles.append(headlines[i].get_text())
    return news_titles

In [7]:
#This function finds the date and time of publishing of the article.
def date_and_time(links):
    date=[]
    time=[]
    for i in range(len(links)):
        content_data=requests.get(links[i],'lxml')
        soup1=bs(content_data.content,'html5lib')
        when=soup1.find('meta',{"name":"publish-date"})
        date_and_time=when['content'].split('T')
        date.append(date_and_time[0])
        time.append(date_and_time[1].replace("+05:30"," IST"))
    return date,time

In [6]:
#this function goes to the links of the headines and gets the full content
def retrieving_news_content(links):
    news_contents=[]
    for i in range(len(links)):
        content_data=requests.get(links[i],'lxml')
        soup1=bs(content_data.content,'html5lib')
        raw_content=soup1.findAll('p',class_="")
        paragraph=''
        for j in range(len(raw_content)):
            if len(raw_content[j].get_text())>10:
                paragraph+=raw_content[j].get_text()
        news_contents.append(paragraph.strip('\n').split("Start your 14 days trial now.")[0])
    return news_contents

In [ ]:
#Enter the start and end date for news articles in yyyy-mm-dd format.
start_date=date(2010,5,4)
end_date=date(2010,5,6)
time_difference=(end_date-start_date).days

In [8]:
#This is the function to retrieve the articles and make a corpus.
def news_articles():
    all_links=[]
    all_news_titles=[]
    all_authors=[]
    all_noauthors=[]
    all_news_contents=[]
    all_dates=[]
    all_times=[]
    c=0
    for i in range(0,time_difference):
        year=(start_date+timedelta(i)).year
        month=(start_date+timedelta(i)).month
        day=(start_date+timedelta(i)).day
        information=retrieving_information_for_the_day(year,month,day)
        headlines=headlines(information)
        links=links(headlines)
        authors,noauthors=authors(links)
        authors,links,headlines=processing(authors,noauthors,links,headlines)
        all_authors.append(authors)
        all_links.append(links)
        news_titles=news_titles(headlines)
        all_news_titles.append(news_titles)
        date,time=date_and_time(links)
        all_date.append(date)
        all_times.append(time)
        news_contents=retrieving_news_content(links)
        all_news_contents.append()
    return all_links,all_news_titles,all_authors,all_news_contents,all_dates,all_times